# ADAG solver

MANET comes with implementation of ADAG algorithm which solves the MAP inference of a generix Markov Network. The ADAG algorihm solves a discrete optimization (so called max-sum) problem 
$$
    \hat{\mathbf{y}} \in arg\max_{\mathbf{y}\in\cal{Y}^{\cal{V}}}\left ( \sum_{v\in\cal{V}} q_v(y_v)+ \sum_{v,v'\in\cal{E}} g_{idx(v,v')}(y_v,y_{v'})\right ) 
$$
where $\cal{V}=\{0,\ldots,n_V-1\}$ is a set of objects, $\cal{E}\subset \left ( \cal{V}\atop 2\right )$ a set of interacting objects and $\cal{Y}=\{0,1,\ldots,n_Y-1\}$ a set of labels. The unary scores $q_v\colon\cal{Y}\rightarrow\Re$ are represented by NP array $Q [n_Y \times n_V]$. The pair-wise scores $g_i\colon\cal{Y}\times\cal{Y}\rightarrow\Re$, $i=1,\ldots,n_G$, are represented by NP array $G [n_Y \times n_Y \times n_G]$, where $n_G$ is the number of different score functions (e.g. in case of homogeneous MN all edges have the same score and $n_g=0$). The edges are represented by NP array $E [3 \times |\cal{E}|]$ where column $E(:,e)=(v,v',idx(e))$ defines edge $e$ =(v,v')$ conecting objects $v$ and $v'$ and it assignes the $idx(e)$-th pair-wise score function $g_{idx(e)}$ to that edge. The solver returns the maximizing labels $\hat{\mathbf{y}}$ and the value of the objective evaluated at $\hat{\mathbf{y}}$. The ADAG algorithm allows the undirected graph $(\cal{V},\cal{E})$, defining the label interactions, to be arbitrary. 

The ADAG solver has the following syntax:
```
labels, energy = adag( Q, G, E )

Inputs:
    Q [nY x nV] unary functions
    G [nY x nY x nG ] pair functions
    E [3 x nE] edges between objects
Returns:
    labels [nV] maximal labelling
    energy [float]
```

In case, the graph $(\cal{V},\cal{E})$ is a chain, the MAP inference can be solved efficiently by Viterbi algorihtm which has the following syntax:
```
label, energy = viterbi( Q, G )

Inputs:
    Q [nY x nV] unary functions
    G [(nV-1) x nK x nK] pair functions
Returns:
    labels [nV] maximal labelling 
    energy [float] 
```

# Simple example of MAP inference on a chain

Assume the following MAP problem on a chain connecting 3 objects:
$$
   (\hat{y}_0,\hat{y_1},\hat{y_2}) \in arg\max_{y_0,y_1,y_2\in\{0,1\}} \left ( q_0(y_0) + q_1(y_1) + q_2(y_2)+ g(y_0,y_1) + g(y_1,y_2) \right )
$$
where with the unary scores
$$
q_0(0) = 3, q_0(1)=-1, q_1(0)=2, q_1(1)=2, q_2(0)=2, q_2(1)=0
$$
and pair-wise score
$$
   g(0,0) = -1, g(1,0)=2, g(1,1)=4, g(1,1)=-2
$$
The MAP inference can be solved by Viterbi algorithm or the ADAG solver as follows:

In [3]:
import numpy as np
from manet.maxsum import adag
from manet.maxsum import viterbi

Q = np.array([[3, 2, 2], [-1,2,0]])
G = np.array( [[-1,4],[2,-2]] )
E = np.array( [[0,1],[1,2],[0,0]] )

labels, energy = adag( Q, G, E )
print(f"energy={energy}")
print( labels )

labels, energy = viterbi( Q, G)
print(f"energy={energy}")
print( labels )


energy=13.000000100000003
[0 1 0]
energy=13.0
[0 1 0]


# Sudoku solver

Solving Sudoku puzzle can be formulated as MAP inference in appropriately constructed Markov Network which looks as follows:
$$
    \hat{\mathbf{y}} \in arg\max_{\mathbf{y}\in\cal{Y}^{\cal{V}}}\left ( \sum_{v\in\cal{V}} q(x_vy_v)+ \sum_{v,v'\in\cal{E}} g(y_v,y_{v'})\right )
$$
where $\cal{V}=\{ (i,j)\in\N^2\mid 1\leq i \leq 9, 1\leq j \leq 9 \}$ cells of $9\times 9$ grid, $\mathbf{x}=(x_v\in \{?,1,\ldots,9\} \mid v\in\cal{V})$ is the puzzle assignment, $\mathbf{y}=(y_v\in \{1,\ldots,9\} \mid v\in\cal{V})$ the puzzle solution, $\cal{E}$ are edges connecting all cells in each columns, each row, and each $3\times 3$ subgrid and the unary scores are
$$
   q(x,y) = \left \{ \begin{array}{rl} 
    -1 & if\; x \neq ? \land y\neq x\\
    0 & otherwise
   \end{array} \right .
$$
and the pair-wise scores are
$$
   g(y,y') = \left \{ 
    \begin{array}{rl}
      0 & if\; y\neq y' \\
      -1 & if \; y = y'
    \end{array}
    \right . 
$$

Load Sudoku assignments and solutions from a CSV file. 

In [7]:
import csv
import numpy as np

with open('ecml2022/data/sudoku10000.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    sudoku = []
    cnt = 0
    for row in reader:
        assignment = np.array([char for char in row['quizzes']]).astype(int).reshape(9,9)
        solution = np.array([char for char in row['solutions']]).astype(int).reshape(9,9)
        sudoku.append( {'assignment': assignment, 'solution': solution} )
        if cnt == 10:
            break

print(sudoku[0]['assignment'])
print(sudoku[0]['solution'])

[[0 0 4 3 0 0 2 0 9]
 [0 0 5 0 0 9 0 0 1]
 [0 7 0 0 6 0 0 4 3]
 [0 0 6 0 0 2 0 8 7]
 [1 9 0 0 0 7 4 0 0]
 [0 5 0 0 8 3 0 0 0]
 [6 0 0 0 0 0 1 0 5]
 [0 0 3 5 0 8 6 9 0]
 [0 4 2 9 1 0 3 0 0]]
[[8 6 4 3 7 1 2 5 9]
 [3 2 5 8 4 9 7 6 1]
 [9 7 1 2 6 5 8 4 3]
 [4 3 6 1 9 2 5 8 7]
 [1 9 8 6 5 7 4 3 2]
 [2 5 7 4 8 3 9 1 6]
 [6 8 9 7 3 4 1 2 5]
 [7 1 3 5 2 8 6 9 4]
 [5 4 2 9 1 6 3 7 8]]


Translate Sudoku puzzle into MAP inference in Markov Network, i.e. define unary scores, pair-wise scores and edges of the Markov Network.

In [5]:
from math import ceil

def sudoku_to_mnet( sudoku ):
    K = 1
    Q = -K*np.ones([9,9*9] )
    obj = 0
    for row in range(0,9):
        for col in range(0,9):
            symbol = sudoku[row,col]
            if symbol == 0:
                Q[:,obj] = 0
            else:
                Q[symbol-1,obj] = 0
            obj += 1

    G = -K*np.identity( 9 )
    E = []
    for i1 in range(1,10):
        for j1 in range(1,10):
            for i2 in range(1,10):
                for j2 in range(1,10):
                    if i1==i2 or j1==j2 or (ceil(i1/3)==ceil(i2/3) and ceil(j1/3)==ceil(j2/3)):
                        v0 = i1 + (j1-1)*9 - 1
                        v1 = i2 + (j2-1)*9 - 1
                        if v0 != v1 and ([v0,v1] not in E) and ([v1,v0] not in E):
                            E.append([v0,v1])
    E = np.concatenate( (np.array(E).transpose(), np.zeros((1,len(E)),dtype=int)) ,axis=0 )

    return Q, G, E

Call ADAG algorithm on Sudoku puzzle.

In [8]:
from manet.maxsum import adag

# take i-th puzzle
i = 0
assignment = sudoku[i]['assignment']
correct_solution = sudoku[i]['solution']

# call ADAG solver
Q, G, E = sudoku_to_mnet( assignment )

solution, energy = adag( Q, G, E )
solution = solution.reshape(9,9)+1

# assignment, solution and (possible) errors
print( "energy:", energy )
print( assignment)
print( solution )
print( solution - correct_solution )

energy: 0.0
[[0 0 4 3 0 0 2 0 9]
 [0 0 5 0 0 9 0 0 1]
 [0 7 0 0 6 0 0 4 3]
 [0 0 6 0 0 2 0 8 7]
 [1 9 0 0 0 7 4 0 0]
 [0 5 0 0 8 3 0 0 0]
 [6 0 0 0 0 0 1 0 5]
 [0 0 3 5 0 8 6 9 0]
 [0 4 2 9 1 0 3 0 0]]
[[8 6 4 3 7 1 2 5 9]
 [3 2 5 8 4 9 7 6 1]
 [9 7 1 2 6 5 8 4 3]
 [4 3 6 1 9 2 5 8 7]
 [1 9 8 6 5 7 4 3 2]
 [2 5 7 4 8 3 9 1 6]
 [6 8 9 7 3 4 1 2 5]
 [7 1 3 5 2 8 6 9 4]
 [5 4 2 9 1 6 3 7 8]]
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
